<a href="https://colab.research.google.com/github/aj1365/DeepForestPaper/blob/main/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
######################################### Connecting to Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/DForest

/content/drive/MyDrive/DForest


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
Data = np.loadtxt('DataAvalon2.csv', delimiter=',')

X_train, X_test, y_train, y_test = train_test_split(Data[:,0:27],
                                                    Data[:,28],
                                                    stratify=Data[:,28],
                                                    test_size=0.50,
                                                    random_state=0)

In [ ]:
###################### Training the Random Forest and Decision Tree models

from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

model = RandomForestClassifier(n_estimators=200, random_state=0)
#model= tree.DecisionTreeClassifier()
model.fit(X_train, y_train)
ypred = model.predict(X_test)
ca = np.sum(ypred == y_test) / y_test.shape[0]

print("Classification accuracy: %.5f" % ca)

In [ ]:
############################################# classification metrics

from sklearn import metrics

print(metrics.classification_report(ypred, y_test))

In [ ]:
############################################# classification confusiion matrix

import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

mat = confusion_matrix(y_test, ypred)


#fig, ax = plt.subplots(figsize=(15, 8))
#plot_confusion_matrix(model, X_test, y_test,ax=ax)

#ax.set_title('Confusion Matrix')
mat

In [ ]:
!pip install rasterio

In [ ]:
!pip install geopandas

In [ ]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import numpy as np
from shapely.geometry import mapping
import os

import matplotlib.pyplot as plt
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.windows import Window
from rasterio.plot import reshape_as_raster, reshape_as_image

In [ ]:

study_areafp=rasterio.open('Avalon.tif')
Study_area2=study_areafp.read()

# reshape the image
reshaped_Study_area2=reshape_as_image(Study_area2)
reshaped_Study_area2.shape

In [ ]:
############################################################## Removing nan and infinite values

#reshaped_Study_area2[np.isfinite(reshaped_Study_area2)]=-1
reshaped_Study_area2[np.isnan(reshaped_Study_area2)]=-1.0
#reshaped_Study_area2

In [ ]:
nsamples, nx, ny = reshaped_Study_area2.shape
d2_train_dataset = reshaped_Study_area2.reshape((nsamples*nx,ny))

In [ ]:

class_prediction2=model.predict(d2_train_dataset[:,0:27])

In [ ]:
np.min(class_prediction2), np.max(class_prediction2)

In [ ]:
# Reshape our classification map back into a 2D matrix so we can visualize it
class_prediction2 = class_prediction2.reshape(reshaped_Study_area2[:, :, 0].shape)


In [ ]:
class_prediction2.shape

In [ ]:
################################################ Predicted image

from matplotlib.patches import Patch, colors

fig, ax = plt.subplots(figsize=(10, 8))

cmapCHM = colors.ListedColormap(['lightgreen','darkgreen','yellow','purple','lightblue','red','darkblue','orange'])

chm_plot = ax.imshow(class_prediction2,
                     cmap=cmapCHM)

ax.set_title("Predicted classes")
# Add a legend for labels
legend_labels = {"lightgreen": "Bog", "darkgreen": "Fen", "yellow": "Marsh","purple": "Swamp","lightblue": "Sh-water","red": "Urban","darkblue": "D-water","orange": "Upland"}

patches = [Patch(color=color, label=label)
           for color, label in legend_labels.items()]

ax.legend(handles=patches,
          bbox_to_anchor=(1.35, 1),
          facecolor="white")

ax.set_axis_off()
plt.show()

In [ ]:
!pip install pyrsgis

In [ ]:
from pyrsgis import raster

Warning! matplotlib_scalebar library not found. You may not be able to export map directly.


In [ ]:
ds, featuresAvalon = raster.read('Avalon.tif')

In [ ]:
################################################################################## Exporting Raster Prediction into Google Drive

outFile = '/content/drive/MyDrive/CNN/RF.tif'
raster.export(class_prediction2, ds, filename=outFile, dtype='float')